# Title: Search Engine Alpha 2.0
<p><b>Abstract:</b> Search engine build using Reuters data set</p>
<p><b>Authors:</b> Uriel Antonio & Ernesto Louie Cortez</p>
<p><b>Date:</b>    04/30/2016</p>

In [1]:
from xml.etree import ElementTree
import re
from StringIO import StringIO
from bs4 import BeautifulSoup
import os 
import pandas as pd
from collections import Counter

## Data Cleaning

<p><b>Abstract: </b>Reuters data is organized in a generalized markup format.  Using BeatifulSoup, articles are scraped for their Title, content, date and place acording to their respective markup tags</p>

<p><b>Challenges: </b>1) We initially made the assumption that all articles would have a date, place, title, and body.  During alpha 1 stage, it was found that titles and content became mismatched approximately 30 articles in.  Conditional statements were created to insert placeholders when content was missing.  2) The cleaned data was also littered with mark up tags for special symbols that also needed to be cleaned.    </p>


<p><b></b></p>

In [2]:
totstring=""

with open("Data/reut2-000.sgm",'r') as inF:
    for line in inF:
        string2=re.sub("&.*?>","",line,flags=re.UNICODE)
        string3=re.sub("\n"," ",string2,flags=re.UNICODE)
        string=re.sub("[^0-9a-zA-Z<>/\s=!-\"\"]+","",string3.lower())
        totstring+=string
    
soup= BeautifulSoup(totstring)

items_date=list()
items_places=list()
items_title=list()
items_body=list()


for a in soup.findAll("reuters"):
    if a.date != None:
        items_date.append(a.date.getText())
    else:
        items_date.append("N/D")
    if a.places != None:
        items_places.append(a.places.getText()) 
    else:
        items_places.append("N/L")
    if a.title != None:
        items_title.append(a.title.getText())  
    else:
        items_title.append("Untitled")
    if a.content != None:
        items_body.append(a.content.getText())
    else:
        items_body.append("No Content.")
  

corpus = items_title[0:25]
print(corpus)


[u'bahia cocoa review', u'standard oil  to form financial unit', u'texas commerce bancshares  files plan', u'talking point/bankamerica  equity offer', u'national average prices for farmerowned reserve', u'argentine 1986/87 grain/oilseed registrations', u'red lion inns files plans offering', u'usx  debt dowgraded by moodys', u'champion products  approves stock split', u'computer terminal systems  completes sale', u'cobanco inc  year net', u'ohio mattress  may have lower 1st qtr net', u'am international inc  2nd qtr jan 31', u'brownforman inc  4th qtr net', u'national intergroup to offer permian units', u'economic spotlight  bankamerica ', u'national health enhancement  new program', u'dean foods  sees strong 4th qtr earnings', u'bonus wheat flour for north yemen   usda', u'credit card disclosure bills introduced', u'hughes capital unit signs pact with bear stearns', u'magma lowers copper 075 cent to 66 cts', u'brownforman  sets stock split ups payout', u'esquire radio and electronics in

/home/ubuntu/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
def print_results(results,n, head=True):
    ''' Helper function to print results
    '''
    if head:    
        print('\nTop %d from recall set of %d items:' % (n,len(results)))
        for r in results[:n]:
            print('\t%0.2f - %s'%(r[0],items_title[r[1]]))
    else:
        print('\nBottom %d from recall set of %d items:' % (n,len(results)))
        for r in results[-n:]:
            print('\t%0.2f - %s'%(r[0],items_title[r[1]]))

## Inverted Index

In [5]:
import math
from collections import Counter

def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        for word in doc.split():
            if word in idx:
                if i in idx[word]:
                    idx[word][i] += 1
                else:    
                    idx[word][i] = 1
            else:
                idx[word] = {i:1}
    return idx



## TF-IDF

<p><b>Abstract: </b>Determining the aboutness of a document was improved by using a combined score for both the title and the content of the article.   This assisted in getting articles with numerous term matches in the title to be ranked higher.</p>

<p><b>Challenges: </b> Unresolved issues are synonym matching.  Implementing scrapped date and place data.</p>

In [7]:
def get_results_tfidf(qry, idx_body, n, idx_title, nn):
    score = Counter()
    score2 = Counter()
    for term in qry.split():
        if term in idx_body:
            i = math.log(float(n)/(1+len(idx_body[term])))
            for doc in idx_body[term]:
                score[doc] += idx_body[term][doc] * i
                
        if term in idx_title:
            i = math.log(float(n)/(1+len(idx_body[term])))
            
            for doc in idx_title[term]:
                score2[doc] += idx_title[term][doc]* i
                
    score = score + score2;

    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])
    
    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    #type(score2)
    #print(score2)
    return sorted_results

idx_body = create_inverted_index(items_body)
idx_title= create_inverted_index(items_title)


#results = get_results_tfidf('this is the end of the file', idx_body, len(items_body),idx_title, len(items_title))
#results = get_results_tfidf('japan the oil bankrupt this', idx_body, len(items_body),idx_title, len(items_title))
#results = get_results_tfidf('stock market decline', idx_body, len(items_body),idx_title, len(items_title))
results = get_results_tfidf('japan trade talk', idx_body, len(items_body),idx_title, len(items_title))

print_results(results,10)

{u'portugal"': {831: 1}, u'110000': {227: 1, 996: 1, 94: 1}, u'sowell': {568: 1}, u'yellow': {95: 1}, u'considered"': {306: 1}, u'four': {128: 1, 640: 1, 386: 1, 747: 1, 772: 1, 854: 1, 262: 2, 135: 1, 364: 1, 779: 1, 12: 1, 642: 1, 271: 2, 912: 1, 18: 1, 963: 1, 276: 1, 533: 1, 662: 1, 23: 1, 280: 1, 537: 1, 410: 1, 412: 1, 666: 1, 286: 1, 544: 2, 161: 2, 291: 1, 293: 1, 936: 1, 988: 1, 555: 1, 300: 1, 419: 1, 669: 2, 304: 1, 840: 1, 178: 1, 365: 2, 542: 1, 399: 1, 58: 1, 59: 1, 972: 1, 757: 1, 360: 1, 323: 1, 708: 1, 417: 1, 672: 1, 329: 1, 714: 1, 204: 4, 738: 1, 889: 2, 210: 1, 339: 1, 212: 2, 342: 2, 343: 1, 852: 1, 346: 1, 347: 1, 476: 1, 557: 1, 610: 1, 612: 1, 357: 3, 231: 1, 232: 1, 746: 1, 491: 1, 108: 1, 109: 1, 876: 1, 880: 2, 241: 1, 242: 2, 562: 1, 633: 1, 332: 1, 377: 1, 380: 1, 919: 1}, u'circuitry': {337: 2}, u'travaux': {933: 1}, u'35232': {881: 1}, u'aggression': {236: 1}, u'increase': {900: 1, 261: 2, 134: 1, 391: 1, 8: 1, 9: 1, 139: 1, 972: 2, 144: 1, 273: 1, 402: 